In [1]:
import pandas as pd
import requests
from apikey import apikey 
%load_ext blackcellmagic

In [2]:
# load and transform 2019 data
df_soil_19 = pd.read_excel("2019_Metals_Results_-_Soil_Kitchen.xls")

df_soil_19 = df_soil_19.drop(columns=["Unnamed: 0"])

df_soil_19.columns = df_soil_19.iloc[0]

df_base_19 = df_soil_19.head()

df_soil_19 = df_soil_19.drop(index=[0, 1, 2, 3, 4])

df_soil_19 = df_soil_19[["Street", "Cross Street", "Lead", "Arsenic"]]

In [3]:
# Load and transform 2015-2018 data
df_soil_18_15 = pd.read_csv("2015-18 Soil_Kitchen_Log.csv")

df_soil_18_15 = df_soil_18_15[
    ["Street", "Cross Street", "Full Address", "Lead", "Arsenic"]
]

In [4]:
def request_geocoder(intersection, apikey, city="Austin, TX"):
    """returns lat. lon for cross streets in street1@street2 format.
    Defailt city is Austin, TX and apikey is required"""

    base_url = "https://geocoder.ls.hereapi.com/6.2/geocode.json"
    parameters = {"city": city, "street": intersection, "gen": 9, "apiKey": apikey}
    try:
        response = requests.get(base_url, params=parameters)
        j_resp = response.json()
        lat = j_resp["Response"]["View"][0]["Result"][0]["Location"]["DisplayPosition"][
            "Latitude"
        ]
        lon = j_resp["Response"]["View"][0]["Result"][0]["Location"]["DisplayPosition"][
            "Longitude"
        ]
    #         print(parameters['street'], 'success')
    except:
        lat = ""
        lon = ""
        print(parameters["street"], response)

    return pd.Series([lat, lon], index=["Lat", "Lon"])

In [5]:
def query_cross_streets(df):
    """this function uses geocoder.ls.hereapi.com to find the geocodes for cross streets from a dataframe"""
    df[["Lat", "Lon"]] = df.apply(
        lambda x: request_geocoder(x["Intersection"], apikey), axis=1
    )
    return df

In [6]:
df_soil_18_15["Lead"] = pd.to_numeric(df_soil_18_15["Lead"], errors="coerce")

df_soil_18_15["Arsenic"] = pd.to_numeric(df_soil_18_15["Arsenic"], errors="coerce")

df_soil = df_soil_19.append(df_soil_18_15, ignore_index=True)

df_soil["Intersection"] = df_soil.apply(
    lambda x: str(x["Street"]) + "@" + str(x["Cross Street"]), axis=1
)

# fillna with median values to preserve dataset size
df_soil = df_soil.fillna(df_soil.median())

df_soil_mean = (
    df_soil[["Intersection", "Lead", "Arsenic"]]
    .groupby(["Intersection"])
    .mean()
    .reset_index()
)

# print(df_soil)

In [7]:
# geocode data
df_soil_mean_loc = query_cross_streets(df_soil_mean)

290E@973 <Response [200]>
A.W. Grimes@Thompson Trl <Response [200]>
AW Grimes @ Gattis School <Response [200]>
Anarosa@Scofield Farms <Response [200]>
Barley Field Pass@Dusty Chisolm Trail <Response [200]>
Basket Flower Bend@Prairie Verbena <Response [200]>
Byton Loop@Cherry Creek <Response [200]>
Country Ln@Morning Sun Ln <Response [200]>
Crestwind@Western Sky <Response [200]>
Cushing Dr@School Days Dr <Response [200]>
E. Baker@Moonlit <Response [200]>
Fairweather @ New Hope <Response [200]>
I-H 290 @ McArthur Dr. <Response [200]>
Indian Summer Pass@Apache <Response [200]>
Land Grant Way@Cotton Gin <Response [200]>
Moutainclimb@Dry Creek <Response [200]>
Oakwood Glen Dr. @ Spider Lily <Response [200]>
Oakwood Glen Dr. @ Spider Lily -- ANNEX <Response [200]>
Old Kimbro Rd@Us-290 East <Response [200]>
Paddle Bend Springs@IH35 <Response [200]>
Ray C. Ewry Ln@J Weism <Response [200]>
S. Manchaca@Frate-Barker <Response [200]>
SE Inner Loop@Rockride Ln <Response [200]>
Sandy Bottom @Blue Po

In [8]:
df_soil_mean_loc[['Lat', 'Lon']] = df_soil_mean_loc[['Lat', 'Lon']].fillna(df_soil_mean_loc[['Lat', 'Lon']].median())

In [9]:
df_soil_mean_loc.to_csv('df_soil_mean_loc.csv')